# Weather Predictions on AWS Autopilot with NOAA Data

https://docs.opendata.aws/noaa-ghcn-pds/readme.html

In [17]:
%%sh
apt-get install -y gzip
wget -nv http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt -P ./weather/

wget -nv http://noaa-ghcn-pds.s3.amazonaws.com/csv.gz/2022.csv.gz -P ./weather/
gzip -d ./weather/2022.csv.gz

Reading package lists...
Building dependency tree...
Reading state information...
gzip is already the newest version (1.9-3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


2022-10-29 16:53:29 URL:http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt [10593738/10593738] -> "./weather/ghcnd-stations.txt.1" [1]
2022-10-29 16:53:31 URL:http://noaa-ghcn-pds.s3.amazonaws.com/csv.gz/2022.csv.gz [97608893/97608893] -> "./weather/2022.csv.gz" [1]


Format of the stations file:

Variable	Columns	Type	Example    
ID	1-11	Character	EI000003980    
LATITUDE	13-20	Real	55.3717    
LONGITUDE	22-30	Real	-7.3400    
ELEVATION	32-37	Real	21.0    
STATE	39-40	Character    
NAME	42-71	Character	MALIN HEAD    
GSN FLAG	73-75	Character	GSN    
HCN/CRN FLAG	77-79	Character    
WMO ID	81-85	Character	03980    

In [18]:
import pandas as pd
colspecs = [(0, 11), (12, 20), (21, 30), (31, 37), (38, 40), (41, 71), (72, 75), (76, 79), (80, 85)]

station_data = pd.read_fwf('./weather/ghcnd-stations.txt', header=None, colspecs=colspecs,
                           names=['id', 'latitude', 'longitude', 'elevation', 'state', 'name', 'gsn_flag', 'hcn_crn_flag', 'wmo_id'])
station_data[:10]

,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
5,AEM00041218,24.2620,55.6090,264.9,NaN,AL AIN INTL,NaN,NaN,41218.0
6,AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
7,AFM00040938,34.2100,62.2280,977.2,NaN,HERAT,NaN,NaN,40938.0
8,AFM00040948,34.5660,69.2120,1791.3,NaN,KABUL INTL,NaN,NaN,40948.0
9,AFM00040990,31.5000,65.8500,1010.0,NaN,KANDAHAR AIRPORT,NaN,NaN,40990.0


In [19]:
station_data_for_model = station_data[['state','id']]
station_data_for_model.dropna().to_csv('./weather/stations.csv', index=False)

In [38]:
%%bash

{ echo 'id, date, element, value, M-FLAG, Q-FLAG, S-FLAG, OBS-TIME'; cat ./weather/2022.csv; } > ./weather/2022_with_headers.csv

awk -F '","'  'BEGIN {OFS=","} { if (toupper($6) == ""  && toupper($3) ~ /^WT/)  print }' ./weather/2022_with_headers.csv > ./weather/2022_with_headers_filtered.csv


In [39]:
#awk -F '","'  'BEGIN {OFS=","} { if (toupper($6) == "NaN" && $3 ~ /^WT/)  print }' ./weather/2022_with_headers.csv > ./weather/2022_with_headers_filtered.csv
pd.read_csv('./weather/2022_with_headers_filtered.csv', nrows=100)

EmptyDataError: No columns to parse from file

In [ ]:
%%bash 
{
  echo "id, state, Tmax, Date, Tmin1, Tmax1"
  join -t, <(sort ./weather/2022.csv) <(sed 1d ./weather/stations.csv | sort)
} > ./weather/combined.csv

In [ ]:
import sagemaker

prefix = 'sagemaker/weather-predictions/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="./weather/2022.csv", key_prefix=prefix)
print(uri)